In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [86]:
df = pd.read_csv("./data/regression_2/regression_2.csv")
# df = pd.read_csv("./data/dataset.csv")
df.describe()

,x1,x2,y
count,1000.000000,1000.000000,1000.000000
mean,0.155582,0.320792,65.285132
std,5.770767,5.649911,39.310109
min,-9.882128,-9.999302,0.142260
25%,-4.963285,-4.486459,34.214396
50%,0.352472,0.517567,61.602042
75%,5.357289,5.243427,92.409980
max,9.968177,9.952260,184.233414


In [100]:
def add_features(df):
    result_df = df.copy()
    result_df["feature_x1_mult_x2"] = df.apply(lambda row: row["x1"] * row["x2"], axis=1)
    result_df["feature_x1_mult_x1"] = df.apply(lambda row: row["x1"] * row["x1"], axis=1)
    result_df["feature_x2_mult_x2"] = df.apply(lambda row: row["x2"] * row["x2"], axis=1)    
    return result_df

In [101]:
features_df = add_features(df)

In [102]:
msk = np.random.rand(len(features_df)) < 0.8
train_ds = features_df[msk]
test_ds = features_df[~msk]

In [103]:
def pd_input_fn(df, y_label):
  return tf.estimator.inputs.pandas_input_fn(
      x=df,
      y=df[y_label],
      batch_size = 32,
      num_epochs = 5,
      shuffle = True,
      queue_capacity = 1000,
      num_threads = 1
  )

def pd_predict_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
      x=df,
      batch_size = 32,
      num_epochs = 5,
      shuffle = True,
      queue_capacity = 1000,
      num_threads = 1
  )

In [104]:
feature_columns = [
    tf.feature_column.numeric_column(key="x1", dtype=tf.float32),
    tf.feature_column.numeric_column(key="x2", dtype=tf.float32),
    tf.feature_column.numeric_column(key="feature_x1_mult_x2", dtype=tf.float32),
    tf.feature_column.numeric_column(key="feature_x1_mult_x1", dtype=tf.float32),
    tf.feature_column.numeric_column(key="feature_x2_mult_x2", dtype=tf.float32),
]
estimator = tf.estimator.DNNRegressor(
    hidden_units=[4,2],
    feature_columns=feature_columns,
    model_dir="./models/estimator_regression/"
)
estimator.train(input_fn=pd_input_fn(train_ds, "y"))


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/estimator_regression/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0684a03e10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tenso

In [105]:
estimator.evaluate(input_fn=pd_input_fn(test_ds, "y"))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-25-10:37:19
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/estimator_regression/model.ckpt-253
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-25-10:37:20
INFO:tensorflow:Saving dict for global step 253: average_loss = 0.22899656, global_step = 253, label/mean = 66.30532, loss = 7.2647185, prediction/mean = 66.400734
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 253: ./models/estimator_regression/model.ckpt-253


{'average_loss': 0.22899656,
 'global_step': 253,
 'label/mean': 66.30532,
 'loss': 7.2647185,
 'prediction/mean': 66.400734}

In [107]:
estimate_df = add_features(pd.DataFrame(data={"x1":[2.0], "x2":[4.0]}))
    

predictions = estimator.predict(input_fn=pd_predict_input_fn(estimate_df))
print(predictions)
print(list(predictions))
# print(list(predictions)[0])
    

<generator object Estimator.predict at 0x7f068480c5c8>
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/estimator_regression/model.ckpt-253
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[{'predictions': array([20.58233], dtype=float32)}, {'predictions': array([20.58233], dtype=float32)}, {'predictions': array([20.58233], dtype=float32)}, {'predictions': array([20.58233], dtype=float32)}, {'predictions': array([20.58233], dtype=float32)}]


In [ ]:
estimator.e